<a href="https://colab.research.google.com/github/chrishg23-jpg/HES-benchmark/blob/main/HES2full4fail_at_200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# HES_21.0_DIMENSIONAL_HARMONIC_LOCK.ipynb: Act XXI - Testing N=192 Harmonic
import numpy as np
import warnings
import time

warnings.filterwarnings("ignore", category=RuntimeWarning)
np.random.seed(42)

# --- 1. CORE PARAMETERS & SETUP (N=192 HARMONIC) ---
alpha, beta, gamma = 1.41, 0.0, 0.000  # Beta and Gamma are ZERO (Pure Compensatory)
fixed_delta = 2.54
G_values = [0.1, 1.0, 10.0]
grid_size = 192                         # New Grid Size
steps = 50
step_to_gpc = 0.1
subsystem_size = 3
# Normalization Factor: L^2 = (192/100)^2 = 3.6864
L_scale_factor = (grid_size / 100.0)**2

# --- 2. CORE FUNCTIONS (RETAINED) ---

def von_neumann_entropy(subsystem):
    """Computes von Neumann entropy of a subsystem's density matrix."""
    flat = subsystem.flatten()
    rho = np.outer(flat, flat)
    rho /= np.trace(rho + 1e-10)
    eigenvalues = np.linalg.eigvalsh(rho)
    eigenvalues = eigenvalues[eigenvalues > 1e-10]
    return -np.sum(eigenvalues * np.log2(eigenvalues)) if len(eigenvalues) > 0 else 0.0

def laplacian_2d_numpy(field):
    """OPTIMIZED Laplacian (∇²S) using vectorized NumPy slicing."""
    S_padded = np.pad(field, 1, mode='wrap')
    S_center = S_padded[1:-1, 1:-1]
    L = S_padded[2:, 1:-1] + S_padded[:-2, 1:-1] + S_padded[1:-1, :-2] + S_padded[1:-1, 2:] - 4 * S_center
    return L

def boundary_correlator(field, grid_size):
    """Patched Boundary Correlator for consistent stacking."""
    top = field[0, :]
    bottom = field[-1, :]
    left = field[1:-1, 0]
    right = field[1:-1, -1]

    padding_amount = 1
    left_padded = np.pad(left, (padding_amount, padding_amount), mode='constant', constant_values=0)
    right_padded = np.pad(right, (padding_amount, padding_amount), mode='constant', constant_values=0)

    boundary_data = np.vstack([top, bottom, left_padded, right_padded])
    correlator = np.corrcoef(boundary_data)
    off_diag = correlator[np.triu_indices(4, k=1)]
    return np.mean(np.abs(off_diag)) if off_diag.size > 0 else 0.0

def calculate_metrics(s, entropy_map, C_history, G, a, b, g, d, total_steps, grid_size):
    """Post-analysis calculations: G_eff, lambda_sim, Correlator, G_sweep."""

    # 1. Estimate effective G
    boundary_area = 4 * grid_size
    boundary_entropy = np.mean([entropy_map[0,:], entropy_map[-1,:],
                                entropy_map[:,0], entropy_map[:,-1]])
    G_eff = boundary_area / (4 * boundary_entropy) if boundary_entropy > 0 else 1.0
    correlator = boundary_correlator(s, grid_size)

    # 2. Compute Lambda (λ_sim)
    lambda_sim = 0.0
    if len(C_history) >= total_steps:
        fit_start, fit_end = 20, 50
        C_data = np.array(C_history[fit_start:fit_end])
        if C_data.size >= 2 and np.max(C_data) > 1e-18:
            log_C = np.log(C_data + 1e-18)
            time_steps = np.arange(len(log_C))
            poly_coeffs = np.polyfit(time_steps, log_C, 1)
            lambda_sim = -poly_coeffs[0]

    # 3. Test G-values for Curvature Sensitivity (Short 5-step run)
    G_curvature = {}
    for G_val in G_values:
        s_G = np.copy(s)
        local_curvature_history = []
        for _ in range(5):
            for i in range(grid_size):
                for j in range(grid_size):
                    i_start, i_end = max(0, i - subsystem_size//2), min(grid_size, i + subsystem_size//2 + 1)
                    j_start, j_end = max(0, j - subsystem_size//2), min(grid_size, j + subsystem_size//2 + 1)
                    if i_end > i_start and j_end > j_start:
                        entropy_map[i,j] = von_neumann_entropy(s_G[i_start:i_end, j_start:j_end])

            curvature_G_norm = laplacian_2d_numpy(entropy_map) / L_scale_factor
            local_curvature_history.append(np.mean(np.abs(curvature_G_norm)))

            nabla_s_approx = (s_G - (np.roll(s_G, 1, axis=0) + np.roll(s_G, -1, axis=0) +
                                   np.roll(s_G, 1, axis=1) + np.roll(s_G, -1, axis=1)) / 4.0)
            s_G = np.clip(s_G - (a * nabla_s_approx + b * s_G + g * np.mean(s_G) * s_G + d * curvature_G_norm), -1, 1)

        G_curvature[G_val] = np.mean(local_curvature_history)

    return {
        'G_eff': G_eff,
        'lambda_sim': lambda_sim,
        'gamma_gw_sim': lambda_sim / (total_steps * step_to_gpc),
        'correlator': correlator,
        'G_curvature': G_curvature
    }

def run_simulation(grid_size, G, a, b, g, d, total_steps):
    start_time = time.time()
    s = np.random.uniform(-1, 1, (grid_size, grid_size))
    entropy_map = np.zeros((grid_size, grid_size))
    C_history = []

    print(f"Starting {grid_size}x{grid_size} DIMENSIONAL HARMONIC LOCK (α={a:.2f}, δ={d:.2f}, L^2 Norm Factor={L_scale_factor:.4f}).")

    # MAIN TIME EVOLUTION LOOP
    for step in range(total_steps):
        # 1. Compute Entropy Map
        for i in range(grid_size):
            for j in range(grid_size):
                i_start, i_end = max(0, i - subsystem_size//2), min(grid_size, i + subsystem_size//2 + 1)
                j_start, j_end = max(0, j - subsystem_size//2), min(grid_size, j + subsystem_size//2 + 1)
                if i_end > i_start and j_end > j_start:
                    entropy_map[i,j] = von_neumann_entropy(s[i_start:i_end, j_start:j_end])

        # 2. Compute Curvature and APPLY NORMALIZATION
        curvature = laplacian_2d_numpy(entropy_map)
        curvature_norm = curvature / L_scale_factor

        # 3. Update Rule: s’ = s - α∇s + βs + γ⟨s⟩s + δ(∇²S / L²)
        nabla_s_approx = (s - (np.roll(s, 1, axis=0) + np.roll(s, -1, axis=0) +
                               np.roll(s, 1, axis=1) + np.roll(s, -1, axis=1)) / 4.0)
        s_new = s - (a * nabla_s_approx + b * s + g * np.mean(s) * s + d * curvature_norm)
        s = np.clip(s_new, -1, 1)

        C_history.append(np.mean(np.abs(curvature_norm)))

    elapsed_time = time.time() - start_time
    print(f"Finished {grid_size}x{grid_size} in {elapsed_time:.2f} seconds.")

    # Calculate all metrics after the run
    return calculate_metrics(s, entropy_map, C_history, G=1.0, a=alpha, b=beta, g=gamma, d=fixed_delta, total_steps=steps, grid_size=grid_size)


# --- 3. EXECUTION AND FINAL REPORT ---

# Run HES 21.0 Trial
results = run_simulation(grid_size, G=1.0, a=alpha, b=beta, g=gamma, d=fixed_delta, total_steps=steps)

# --- 4. FINAL REPORT OUTPUT ---

print("\n" + "="*90)
print("HES 21.0 DIMENSIONAL HARMONIC LOCK REPORT (192x192 Grid, Pure Dynamics)")
print(f"PARAMS: N={grid_size}, α={alpha:.2f}, δ={fixed_delta:.2f}, ρ (δ/α)={fixed_delta/alpha:.2f}, β={beta}, γ={gamma:.3f}, L^2={L_scale_factor:.4f}")
print("==========================================================================================")

print("\nACTUAL HES 21.0 RESULTS:")
print("-" * 25)
print(f"1. Boundary Correlator: {results['correlator']:.4f} (Target: 0.81 ± 0.01)")
print(f"2. Estimated Effective G: {results['G_eff']:.4f} (Target: ~1.2)")
print(f"3. Curvature Decay Constant (λ_sim): {results['lambda_sim']:.6f} per step (Target: >0, ~0.07)")
print(f"4. Mapped GW Damping Rate (Γ_GW): {results['gamma_gw_sim']:.6f} per Gpc (Target: ~0.0002)")
print("\nPIE/SPOCK REVIEW: Curvature vs. Input G")
for G, C in results['G_curvature'].items():
    print(f"  G={G:.1f}: Mean |Curvature| = {C:.6e}")
print("-" * 90)
print("ASSESSMENT: Awaiting data from the 192x192 Dimensional Harmonic Lock...")


Starting 192x192 DIMENSIONAL HARMONIC LOCK (α=1.41, δ=2.54, L^2 Norm Factor=3.6864).
Finished 192x192 in 83.06 seconds.

HES 21.0 DIMENSIONAL HARMONIC LOCK REPORT (192x192 Grid, Pure Dynamics)
PARAMS: N=192, α=1.41, δ=2.54, ρ (δ/α)=1.80, β=0.0, γ=0.000, L^2=3.6864

ACTUAL HES 21.0 RESULTS:
-------------------------
1. Boundary Correlator: 0.3495 (Target: 0.81 ± 0.01)
2. Estimated Effective G: 1225653487222.9766 (Target: ~1.2)
3. Curvature Decay Constant (λ_sim): 0.012513 per step (Target: >0, ~0.07)
4. Mapped GW Damping Rate (Γ_GW): 0.002503 per Gpc (Target: ~0.0002)

PIE/SPOCK REVIEW: Curvature vs. Input G
  G=0.1: Mean |Curvature| = 4.327783e-12
  G=1.0: Mean |Curvature| = 4.327783e-12
  G=10.0: Mean |Curvature| = 4.327783e-12
------------------------------------------------------------------------------------------
ASSESSMENT: Awaiting data from the 192x192 Dimensional Harmonic Lock...
